In [1]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)
warnings.filterwarnings("ignore", message=r"cannot", category=UserWarning)

In [2]:
import os
import sys
sys.path.append('..')

from carla.recourse_methods import Dice, Wachter
from copy import deepcopy
from model_shifts import (DynamicCsvCatalog,
                          DynamicOnlineCatalog,
                          RecourseExperiment,
                          RecourseGenerator,
                          train_model,
                          plot_experiment,
                          generate_gif)

os.environ["CUDA_VISIBLE_DEVICES"] = ""

#     torch.manual_seed(0)
#     random.seed(0)
#     np.random.seed(0)

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [3]:
# Load the dataset from a csv file
# TODO: Iris? Penguins? Breast Cancer (categorical)?
dataset = DynamicCsvCatalog(file_path="../datasets/linearly_separable.csv", 
                            continuous=['feature1', 'feature2'], categorical=[],
                            immutables=[], target='target', test_size=0.3)
# dataset = DynamicOnlineCatalog("compas", 'score')

[INFO] Balance: train set 0.5, test set 0.5 [dynamic_csv_catalog.py __init__]


In [4]:
hyper_params = {'learning_rate': 0.01, 'epochs': 4, 'batch_size': 1, 'hidden_size': [8, 4]}

# Recourse generated by DICE is compared with the Wachter generator, as they may modify data differently
# we need to keep track of two models and two datasets and update them independently
model = train_model(dataset, hyper_params)

generators = [
              RecourseGenerator('DICE_1', deepcopy(dataset), deepcopy(model),
                                Dice, {"num": 1}, hyper_params, 30),
            #   RecourseGenerator('DICE_2', deepcopy(dataset), deepcopy(model),
            #                     Dice, {"num": 2}, hyper_params, 30),
              RecourseGenerator('Wachter', deepcopy(dataset), deepcopy(model),
                                Wachter, {"loss_type": "BCE", "t_max_min": 3}, hyper_params, 180)
             ]

# SET NAME BEFORE THE EXPERIMENT
name='linearly_separable_1_ce_per_epoch'
experiment = RecourseExperiment(dataset, model, generators, name)
experiment.run(epochs=70, recourse_per_epoch=1, calculate_p=True)
experiment.save_data()

[INFO] Training the initial model [recourse_generator.py train_model]
[INFO] Updating the DICE_1 model [recourse_generator.py update_model]
[INFO] Retraining the model [recourse_generator.py train_model]
[INFO] Current balance: train set 0.5, test set 0.5 [dynamic_mlmodel_catalog.py retrain]
[INFO] Updating the Wachter model [recourse_generator.py update_model]
[INFO] Retraining the model [recourse_generator.py train_model]
[INFO] Current balance: train set 0.5, test set 0.5 [dynamic_mlmodel_catalog.py retrain]
[INFO] Starting epoch 1 [recourse_experiment.py run]
[INFO] Applying the DICE_1 generator. [recourse_generator.py apply_recourse_with_timeout]
[INFO] Generating counterfactual 1 with DICE_1 [recourse_generator.py apply_recourse_with_timeout]
[INFO] Updating the DICE_1 model [recourse_generator.py update_model]
[INFO] Retraining the model [recourse_generator.py train_model]
[INFO] Current balance: train set 0.5035714285714286, test set 0.5 [dynamic_mlmodel_catalog.py retrain]
[IN

In [5]:
experiment_path = f'../experiment_data/{experiment.experiment_name}'
for g in generators:
    generate_gif(experiment_path, g.name)

In [6]:
config = [
    {'type': 'pos_MMD', 'dict_path': ['MMD', 'positive', 'value']},
    {'type': 'neg_MMD', 'dict_path': ['MMD', 'negative', 'value']},
    {'type': 'pos_distance', 'dict_path': ['decisiveness', 'positive']},
    {'type': 'neg_distance', 'dict_path': ['decisiveness', 'negative']},
    {'type': 'disagreement', 'dict_path': ['disagreement']},
    {'type': 'model_MMD', 'dict_path': ['model_MMD', 'value']}
]

for c in config:
    plot_experiment(experiment_path,
                    [g.name for g in generators],
                    c['type'],
                    c['dict_path'],
                    show_plot=False)

..\model_shifts\plotting\plot_experiment.py:77: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
  plt.ylim([0 - 0.2 * max(result), 1.2 * max(result)])
